# 1. 라이브러리 설치

In [ ]:
import pandas as pd
import os
import numpy as np
import datetime
import warnings
import dask.dataframe as dd
import glob
warnings.filterwarnings(action='ignore')

# 2. 사용자 설정

2-1. default 경로 입력

In [ ]:
default_dir = 'C:/19Project/05.TRV/01.Pred_Demand/'

2-2. 파일 경로 지정

In [ ]:
##### 파일 경로 설정 #####
pred_dir = default_dir + '4. 배차 예측_20211103_2/pred'

# 3. 데이터 불러오기

3-1. 디렉터리 생성

In [ ]:
now = datetime.datetime.now().date().strftime('%Y%m%d')

num = 1

p_table_dir = default_dir + '5. Pred Table_' + now

while os.path.exists(p_table_dir + '_' + str(num)):
    num += 1
    
p_table_dir_num = p_table_dir + '_' + str(num) + '/'

os.makedirs(os.path.join(p_table_dir_num))

3-2. 센터 차량 예측 결과 불러오기

In [ ]:
center_files = glob.glob(pred_dir + '/센터차량_*pred.csv')
center_pred_data = dd.read_csv(center_files, encoding='cp949', engine = "python", header = 0, parse_dates = ['date']) 
center_data = center_pred_data.compute()
center_data.reset_index(drop = True, inplace = True)

3-3. 임차 택시 예측 결과 불러오기

In [ ]:
taxi_files = glob.glob(pred_dir + '/임차택시_*pred.csv')
taxi_pred_data = dd.read_csv(taxi_files, encoding='cp949', engine="python", header=0, parse_dates = ['date']) 
taxi_data = taxi_pred_data.compute()
taxi_data.reset_index(drop = True, inplace = True)

# 4. 함수 생성

In [ ]:
# Pred Table 생성
def p_table(df, car_type):
    df['day'] = df['date'].apply(lambda x: x.date())
    df['time'] = df['date'].apply(lambda x: x.hour)
    # 날짜별 파일 생성
    for d in df['day'].unique():
        #pred_w_grab 생성
        tmp = df[df['day'] == d]
        table_day = tmp[['승차 구역명','time', 'conf_low', 'preds', 'conf_high']]
        table_day.columns = ['승차 구역명','time', '1_low', '2_preds', '3_high']
        pivot_grab = pd.pivot_table(table_day, index = '승차 구역명', columns = 'time', 
                                     values = ['1_low', '2_preds', '3_high'], aggfunc = np.sum, margins = True, fill_value = 0)
        pivot_grab.columns.names = ["value", "time"]
        pivot_grab = pivot_grab.swaplevel("value", "time", 1)
        pivot_grab.sort_index(axis = 1, level = 0, inplace = True)
        pivot_grab.index.name = None
        pivot_grab.columns.names = (None, None)
        pivot_grab.columns.set_levels(['low', 'preds', 'high'],level=1,inplace=True)
        pivot_grab.to_csv(p_table_dir_num + car_type + '_' + str(d) + '_' + 'pred_w_grab.csv', encoding = 'MS949')
        print('[pred_' + str(d) + '] \n' + p_table_dir_num + car_type + '_' + str(d) + '_' + 'pred_w_grab.csv')
        #pred_table 생성
        table_day.columns = ['승차 구역명','time', 'low', 'preds', 'high']
        pivot_pred = pd.pivot_table(table_day, index = '승차 구역명', columns = 'time', 
                                     values = ['preds'], aggfunc = np.sum, margins = True, fill_value = 0)      
        pivot_pred.index.name = None
        pivot_pred.columns.name = None
        pivot_pred.to_csv(p_table_dir_num + car_type + '_' + str(d) + '_' + 'pred_pred.csv', encoding = 'MS949')
        print(p_table_dir_num + car_type + '_' + str(d) + '_' + 'pred_pred.csv \n')

# 5. Pred Table 생성

5-1. 센터 차량

In [ ]:
p_table(center_data, '센터차량')

[pred_2021-08-06] 
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/센터차량_2021-08-06_pred_w_grab.csv
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/센터차량_2021-08-06_pred_pred.csv 

[pred_2021-08-05] 
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/센터차량_2021-08-05_pred_w_grab.csv
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/센터차량_2021-08-05_pred_pred.csv 

[pred_2021-08-04] 
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/센터차량_2021-08-04_pred_w_grab.csv
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/센터차량_2021-08-04_pred_pred.csv 

[pred_2021-08-02] 
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/센터차량_2021-08-02_pred_w_grab.csv
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/센터차량_2021-08-02_pred_pred.csv 

[pred_2021-08-03] 
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/센터차량_2021-08-03_pred_w_grab.csv
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/센터차량_2021-08-03_p

5-2. 임차 택시

In [ ]:
p_table(taxi_data, '임차택시')

[pred_2021-08-06] 
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/임차택시_2021-08-06_pred_w_grab.csv
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/임차택시_2021-08-06_pred_pred.csv 

[pred_2021-08-05] 
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/임차택시_2021-08-05_pred_w_grab.csv
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/임차택시_2021-08-05_pred_pred.csv 

[pred_2021-08-04] 
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/임차택시_2021-08-04_pred_w_grab.csv
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/임차택시_2021-08-04_pred_pred.csv 

[pred_2021-08-02] 
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/임차택시_2021-08-02_pred_w_grab.csv
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/임차택시_2021-08-02_pred_pred.csv 

[pred_2021-08-03] 
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/임차택시_2021-08-03_pred_w_grab.csv
C:/19Project/05.TRV/01.Pred_Demand/5. Pred Table_20211103_1/임차택시_2021-08-03_p

NameError: name 'version' is not defined